In [1]:
%env HADOOP_HOME /usr/lib/hadoop
%env HADOOP_CONF_DIR /etc/hadoop/conf
%env HADOOP_YARN_HOME /usr/lib/hadoop
%env HADOOP_MAPRED_HOME /usr/lib/hadoop
%env HADOOP_HDFS_HOME /usr/lib/hadoop-hdfs

%env SPARK_HOME /usr/lib/spark
%env SPARK_CONF_DIR /etc/spark/conf
#%env PYTHONPATH /opt/spark/python/lib/py4j-0.10.7-src.zip
%env PYSPARK_PYTHON /opt/conda/bin/python
%env PYSPARK_DRIVER_PYTHON /opt/conda/bin/python
%env PYTHONPATH /usr/lib/spark/python/lib/py4j-0.10.7-src.zip
#%env PATH /usr/lib/hadoop/bin:/bin:/usr/lib/spark/bin::/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/tmp:/opt/oracle/instantclient_12_1:/home/ubuntu/.local/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/share/scala/bin:/opt/oracle/instantclient_12_1:/home/ubuntu/.local/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/hadoop-hdfs/bin:/opt/hadoop/bin
!hdfs dfs -rm -r /user/dominospark/example*
!echo $PYTHONPATH

env: HADOOP_HOME=/usr/lib/hadoop
env: HADOOP_CONF_DIR=/etc/hadoop/conf
env: HADOOP_YARN_HOME=/usr/lib/hadoop
env: HADOOP_MAPRED_HOME=/usr/lib/hadoop
env: HADOOP_HDFS_HOME=/usr/lib/hadoop-hdfs
env: SPARK_HOME=/usr/lib/spark
env: SPARK_CONF_DIR=/etc/spark/conf
env: PYSPARK_PYTHON=/opt/conda/bin/python
env: PYSPARK_DRIVER_PYTHON=/opt/conda/bin/python
env: PYTHONPATH=/usr/lib/spark/python/lib/py4j-0.10.7-src.zip
Deleted /user/dominospark/example
/usr/lib/spark/python/lib/py4j-0.10.7-src.zip


In [2]:
#https://saagie.zendesk.com/hc/en-us/articles/360029759552-PySpark-Read-and-Write-Files-from-HDFS
#ON EMR FETCH /usr/lib/hadoop-lzo/lib/
#spark.executor.extraClassPath /etc/hadoop/conf:/etc/hive/conf:/usr/lib/hadoop-lzo/lib/*:/usr/share/aws/aws-java-sdk/*:/usr/share/aws/emr/emrfs/conf:/usr/share/aws/emr/emrfs/lib/*:/usr/share/aws/emr/emrfs/auxlib/*

#spark.driver.extraClassPath /etc/hadoop/conf:/etc/hive/conf:/usr/lib/hadoop-lzo/lib/*:/usr/share/aws/aws-java-sdk/*:/usr/share/aws/emr/emrfs/conf:/usr/share/aws/emr/emrfs/lib/*:/usr/share/aws/emr/emrfs/auxlib/*
#%env SUBMIT_ARGS '--jars /opt/spark/conf/hadoop-lzo-0.4.19.jar'
from pyspark.sql import SparkSession


In [3]:
sparkSession = SparkSession.builder.appName("example-pyspark-read-and-write") \
.master('local[*]') \
.config("fs.default.name", "hdfs://10.0.1.76:8020") \
.getOrCreate()


In [4]:
sparkSession

In [5]:
import random
#Create datasets
data = []
cnt=10000
for i in range(cnt):
   data.append((i,random.randint(1,100),random.randint(1,100),random.randint(1,100)))

In [6]:
hdfs_path='/user/dominospark/example'
df = sparkSession.createDataFrame(data)

# Write into HDFS
df.write.csv(hdfs_path)

In [7]:
df_load = sparkSession.read.csv(hdfs_path)
df_load.show()

+---+---+---+---+
|_c0|_c1|_c2|_c3|
+---+---+---+---+
|  0| 38| 58| 19|
|  1| 95| 56| 16|
|  2| 74|  5| 62|
|  3| 94| 67| 70|
|  4| 14|  3| 94|
|  5| 57| 48| 27|
|  6| 24| 66| 22|
|  7| 67| 83| 84|
|  8| 62| 14| 55|
|  9| 19| 41| 62|
| 10| 83| 10| 72|
| 11| 18| 97| 70|
| 12| 33| 41| 57|
| 13|  2| 37| 13|
| 14| 29| 71| 61|
| 15| 48| 96| 81|
| 16| 22| 64| 32|
| 17|  7| 15|  5|
| 18| 47| 16| 96|
| 19| 52| 43| 15|
+---+---+---+---+
only showing top 20 rows



In [8]:
sparkSession.stop()

In [9]:
!hdfs dfs -ls /user/dominospark

Found 4 items
drwxr-xr-x   - dominospark dominospark          0 2021-08-12 21:37 /user/dominospark/.sparkStaging
drwxr-xr-x   - dominospark dominospark          0 2021-08-12 22:03 /user/dominospark/example
drwxr-xr-x   - dominospark dominospark          0 2021-08-12 21:36 /user/dominospark/mypi
drwxr-xr-x   - dominospark dominospark          0 2021-08-11 15:36 /user/dominospark/test
